In [2]:
import cv2
import torch
import numpy as np
from deep_sort_realtime.deepsort_tracker import DeepSort
from ultralytics import YOLO
import time

In [3]:
model_path = 'best.pt' 
tracker = DeepSort(max_age=30)

In [4]:
model = YOLO(model_path)

In [5]:

video_path = '15sec_input_720p.mp4'  
cap = cv2.VideoCapture(video_path)

output_path = 'output.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    
    results = model(frame, verbose=False)
    detections = results[0].boxes.data.cpu().numpy()

   
    formatted_dets = []
    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        if int(cls) == 0:  # Class 0 = player
            bbox = [x1, y1, x2 - x1, y2 - y1]  # x, y, w, h
            formatted_dets.append((bbox, conf, 'player'))

    
    tracks = tracker.update_tracks(formatted_dets, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {track_id}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

  
    out.write(frame)

  
    cv2.imshow('Player Re-ID', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
cap.release()
out.release()
cv2.destroyAllWindows()